In [2]:
import pandas as pd
# from sunpy.coordinates.transformations import hgs_to_hcc
# from sunpy.coordinates import frames
import sunpy.coordinates
from astropy.coordinates import SkyCoord
from math import isnan
import astropy.units as u


In [11]:
data = pd.read_csv("modified_data.csv")
data.dropna(axis = 0, subset = ["hc_x", "hc_y", "hc_z"], inplace = True)
data.tail()
# data.iloc[9999]

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,HARPNUM,T_REC,NOAA_AR,LAT_FWT,LON_FWT,...,any_flare_in_12h,X_flare_in_24h,M_flare_in_24h,any_flare_in_24h,X_flare_in_48h,M_flare_in_48h,any_flare_in_48h,hc_x,hc_y,hc_z
271300,271300,271300,271300,271300,271300,6683,2016-07-22 13:00:00,12569,16.322458,17.355923,...,0,0,0,0,0,0,0,199112.416410,138678.363837,651808.204228
271301,271301,271301,271301,271301,271301,6683,2016-07-22 14:00:00,12569,16.333773,17.844606,...,0,0,0,0,0,0,0,204527.059410,138923.276145,650077.213397
271302,271302,271302,271302,271302,271302,6683,2016-07-22 15:00:00,12569,16.341875,18.416874,...,0,0,0,0,0,0,0,210853.637271,139161.316217,648001.735941
271303,271303,271303,271303,271303,271303,6683,2016-07-22 16:00:00,12569,16.343138,18.915871,...,0,0,0,0,0,0,0,216359.033051,139299.463555,646154.630359
271304,271304,271304,271304,271304,271304,6683,2016-07-22 17:00:00,12569,16.358110,19.491558,...,0,0,0,0,0,0,0,222674.630444,139631.287568,643933.607255


In [34]:
dlat = data.loc[1, "LAT_FWT"]
dlon = data.loc[1, "LON_FWT"]
print(dlon, dlat)

hgs = SkyCoord(dlon*u.deg, dlat*u.deg, obstime = data.loc[1,"T_REC"], frame = "heliographic_stonyhurst")
hc = hgs.transform_to("heliocentric")
print(hc)
hc.x.value

-1.04546 20.684654
<SkyCoord (Heliocentric: obstime=2010-05-05 04:00:00, observer=<HeliographicStonyhurst Coordinate (obstime=2010-05-05 04:00:00): (lon, lat, radius) in (deg, deg, AU)
    (0., -3.76360511, 1.00848817)>): (x, y, z) in km
    (-11872.01845772, 287843.71407744, 633037.46295764)>


-11872.018457719181